In [1]:
from enum import Enum
from dataclasses import dataclass

In [18]:
from multiprocessing import Pipe

class BufferIdConnection(Pipe):

    def __init__(self) -> None:
        super().__init__()

    

TypeError: method expected 2 arguments, got 3

In [15]:
len(BufferState)

2

In [9]:
a.free

123

In [10]:
BufferState.free

1

In [2]:

class BufferProcess(Process):

    
    def __init__(self, buffer_shape, buffers, message_queue, buffer_id_message_key='buffer_id', stop_message='STOP'):
        super().__init__()
        self.daemon = True
        self._process_id = os.getpid()
        self._buffer_shape = buffer_shape
        self._buffers = buffers
        self._message_queue = message_queue
        self._buffer_id_message_key = buffer_id_message_key
        self._stop_message = stop_message
        
        
    def run(self):
        for message in iter(self._message_queue.get, self.stop_message):
            buffer_id = message[self.buffer_id_message_key]
            data = self.create_data(message)
            buffer = np.ndarray(data.shape, dtype=data.dtype, buffer=self._buffers[buffer_id].buf)
            buffer[:] = data[:]

    @abstractmethod
    def create_data(self, message):
        ...
            
class Data:
    
    def create_data(self, message):
        value = message['value']
        shape = message['shape']
        return np.ones(shape)*value

class DataBufferProcess(Data, BufferProcess):
    ...

In [3]:
number_of_buffers = 2
buffer_shape = (12, 284, 284, 3)

message_queue = Queue()
smm = SharedBufferManager(number_of_buffers=number_of_buffers, buffer_shape=buffer_shape)
smm.start() 
cp1 = DataBufferProcess(buffer_shape=buffer_shape, buffers=smm.buffers, message_queue=message_queue)
cp2 = DataBufferProcess(buffer_shape=buffer_shape, buffers=smm.buffers, message_queue=message_queue)
cp1.start()
cp2.start()





In [12]:
message_queue.put({'buffer_id': 1, 'value': 182, 'shape': buffer_shape})

In [15]:
my_array = np.ndarray(buffer_shape, buffer=smm.get_buffer(1).buf)

In [19]:
np.all(my_array==181)

False

In [ ]:
cp1.join()
cp2.join()
smm.shutdown()